In [31]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [32]:
# Import Lists of Stocks

stocks = pd.read_csv('sp_500_stocks.csv')

In [33]:
# Acquiring an API Token

from secrets import IEX_CLOUD_API_TOKEN

In [34]:
# Making First API Call

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [35]:
# Parsing API Call

price = data['latestPrice']
market_cap = data['marketCap']

In [36]:
# Adding Our Stocks Data to a Panda DataFrame

my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
        index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,155.58,2531243605802,N/A


In [37]:
# Loop through tickers in our list of Stocks

final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
        ignore_index=True
    )

In [38]:
# Using Batch API Calls to Improve Performance

def chunks(lst, n):
    """Yield succcessive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [41]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],index = my_columns),
            ignore_index=True
        )

In [70]:
# Calculating the Number of Shares to buy

portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease Try again")
    portfolio_size = input('Enter the value of your portfolio: ')

Enter the value of your portfolio: 
That's not a number! 
Please Try again
Enter the value of your portfolio: 


In [71]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
print(position_size)
final_dataframe

19801.980198019803


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,176.67,54434473462,112
1,AAL,20.00,12711075659,990
2,AAP,203.36,12970922209,97
3,AAPL,150.36,2485153160990,131
4,ABBV,109.63,195027980467,180
...,...,...,...,...
500,YUM,130.85,38334896828,151
501,ZBH,142.00,29582254652,139
502,ZBRA,587.20,31471768271,33
503,ZION,59.32,9861058534,333


In [82]:
# Formatting Excel Output

writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name ='Recommended Trades', index = False)

# Creating the Formats

background_color = '#0a0a23'
font_color = '#ffffff'


string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [90]:
# # Apply Formats to the Columns of .xlsx File


# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
# writer.save()

In [91]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Numer of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [92]:
# Saving File

writer.save()